## Lets go shopping for Stock Ticker Symbols!

In [1]:
import requests
from retrying import retry
from bs4 import BeautifulSoup
import dill
from tqdm import tqdm_notebook

###### Starting from [BioPharm Catalyst](https://www.biopharmcatalyst.com/calendars/fda-calendar)
We can use requests to pull the site, and bs4 to parse, generating a set of stock ticker symbols in `bpcTickers`

In [2]:
bpcReq = requests.get("https://www.biopharmcatalyst.com/calendars/fda-calendar")

In [3]:
bpcSup = BeautifulSoup(bpcReq.text, "lxml")

In [4]:
bpcTickers = set()
for tag in bpcSup.find_all('a', {'class': 'ticker'}):
    bpcTickers.add(tag.text)

In [5]:
#Well this is every ticker on BPC
print len(bpcTickers)
print bpcTickers

249
set([u'VSAR', u'AGN', u'KITE', u'AAAP', u'NBRV', u'OCUL', u'SAGE', u'BPTH', u'AGTC', u'AGEN', u'MTNB', u'SNDX', u'HSGX', u'MEIP', u'GSK', u'DARE', u'ANIK', u'RETA', u'DBVT', u'VNDA', u'BYSI', u'ONTX', u'GERN', u'CRBP', u'CHMA', u'VRTX', u'PRQR', u'AVIR', u'MYL', u'AZN', u'MBIO', u'AQXP', u'AGLE', u'NVAX', u'DRRX', u'BOLD', u'EXEL', u'SPPI', u'FLXN', u'RVNC', u'XBIT', u'MRK', u'KPTI', u'FGEN', u'PBYI', u'OTIC', u'AMRN', u'ACOR', u'CLRB', u'CEMP', u'KMPH', u'TGTX', u'ANAB', u'TNXP', u'ALRN', u'JNCE', u'JNJ', u'PSTI', u'SNY', u'ARGX', u'AMAG', u'AFMD', u'ARGS', u'GWPH', u'CNAT', u'REGN', u'GBT', u'CRMD', u'LPCN', u'SVRA', u'GTXI', u'ESPR', u'ZFGN', u'ZYNE', u'CLBS', u'LLY', u'KALA', u'PHMMF', u'PTI', u'DCTH', u'IDRA', u'VTVT', u'ARNA', u'SGMO', u'DVAX', u'ALNY', u'SGEN', u'VBLT', u'BMY', u'BHVN', u'EGRX', u'EVOK', u'VRNA', u'BCRX', u'CAPR', u'AXSM', u'GLMD', u'SMMT', u'EYEG', u'CPRX', u'RARE', u'TCON', u'HALO', u'XNCR', u'FBIO', u'NVS', u'RNN', u'FENC', u'AST', u'BTX', u'KURA', u'CBIO

###### Lets now pull a set from another site
How about [valinv](https://www.valinv.com/pdufa/history/1)?
This will require requesting multiple pages

In [6]:
import re

In [7]:
valTickers = set()
valTagPref = re.compile(r'/stock/[A-Z]{1,5}')
#A RegEx for parsing Valinv's links
maxPgNum = 4 
#This should be the highest numbered page to be requested, plus one, from valinv's FDA calendar 
#(at time of writing there were pages 1-3 avalible)
for pgnum in range(1,maxPgNum):
    valReq = requests.get("https://www.valinv.com/pdufa/history/%s" % pgnum)
    valSup = BeautifulSoup(valReq.text, "lxml")
    for tag in valSup.find_all('a', {'href':valTagPref}):
        valTickers.add(tag.text)

So how many ticker symbols have we gathered?

###### Lets pull from another site
How about [RTT News](http://www.rttnews.com/CorpInfo/FDACalendar.aspx?PageNum=1)?

In [8]:
rttTickers = set()
rttTagPref = re.compile(r'http://www\.rttnews\.com/symbolsearch\.aspx\?symbol=[A-Z]{1,5}')
#A RegEx for parsing RTT's links
maxPgNum = 6 
#This should be the highest numbered page to be requested, plus one, from RTT's FDA calendar 
#(at time of writing there were pages 1-5 avalible)
for pgnum in range(1,maxPgNum):
    rttReq = requests.get("http://www.rttnews.com/CorpInfo/FDACalendar.aspx?PageNum=%s" % pgnum)
    rttSup = BeautifulSoup(rttReq.text, "lxml")
    for tag in rttSup.find_all('a', {'href':rttTagPref}):
        rttTickers.add(tag.text)

In [9]:
print(rttTickers)

set([u'XENT', u'AGIO', u'KITE', u'MDCO', u'AMGN', u'PTCT', u'LLY', u'RARE', u'PTLA', u'VRX', u'HALO', u'DVAX', u'TEVA', u'LGND', u'BMY', u'ARRY', u'MYL', u'RHHBY', u'EGRX', u'KMDA', u' AZN', u'GILD', u'JAZZ', u'FLXN', u'KIND', u'NEOS', u'ATRS', u'OCUL', u'TSRO', u'MRK', u'NVS', u'RDUS', u'ADMS', u'PBYI', u'OTIC', u'CELG', u'PFE', u'ABBV', u'IRWD', u'ACRX', u'IPCI', u' AEZS', u' GSK', u'NVO', u'JNJ', u'SGYP'])


## SO?
we've got every ticker symbol from the first three sites I found dealing with PDFUA dates, which companies are represented, and how many follows this title

In [10]:
allTicks = set()
allTicks.update(valTickers)
allTicks.update(bpcTickers)
allTicks.update(rttTickers)

In [11]:
print len(allTicks)

284


In [12]:
print allTicks

set([u'VSAR', u'AGN', u'KITE', u'KALA', u'SAGE', u'BPTH', u'AGEN', u'ESPR', u'XENT', u'RETA', u'IRWD', u'VNDA', u'BYSI', u'CNAT', u'ALDR', u'VRTX', u'AGLE', u'DRRX', u'EXEL', u'FLXN', u'XBIT', u'JNCE', u'MEIP', u'ADMP', u'ADMS', u'OTIC', u'BMY', u'CEMP', u'KMPH', u'ANAB', u'INVA', u'ITEK', u'JNJ', u'PSTI', u' GSK', u'SNY', u'AMAG', u'AFMD', u'GWPH', u'GBT', u'CRMD', u'LPCN', u'SVRA', u'VCEL', u'PTGX', u'ADXS', u'PHMMF', u'DCTH', u'TBPH', u'DVAX', u'SGEN', u'IOVA', u'BHVN', u'NVLS', u'EGRX', u'BCRX', u'AXSM', u'GLMD', u'CPRX', u'RARE', u'HALO', u'NLNK', u'NVS', u'RNN', u'ALDX', u'OPHT', u'KURA', u'TENX', u'NVO', u'OPK', u'EBIO', u'HRTX', u'MTNB', u'MRUS', u'SPPI', u'AMGN', u'CRBP', u'VTL', u'SMMT', u'AZN', u'VRX', u'RDUS', u'MBVX', u'BLRX', u'LOXO', u'CNCE', u'IMGN', u'CORI', u'LJPC', u'MNOV', u'GALE', u'ATRS', u'CLSD', u'INNL', u'CLSN', u' AZN', u'TXMD', u'CHRS', u' AEZS', u'PLX', u'EARS', u'AKBA', u'MYOK', u'OCRX', u'EPZM', u'PGNX', u'CYCC', u'FLKS', u'BLPH', u'ATNM', u'MRNS', u'HSGX'

Lets pickle this for later use, so we can save my poor cloud box some RAM

...we're gonna need it when we start pulling stock prices for all these companies

In [13]:
dill.dump(allTicks, open('Set_of_Ticker_Symbols.pkl', 'w'))